# Transpose Demo

#### Today's Topics
1. Transactions and Blocks
2. NFTs
3. Wallets
4. Fungible Tokens

## Transactions and Blocks

In [ ]:
import os
from transpose import Transpose

# create a new API client with an API key
api = Transpose(os.environ.get('TRANSPOSE_KEY'))

### Transactions - where everything starts

In [ ]:
# Transactions are the core primitive stored on the blockchain. Let's take a look
txns = api.block.transactions_by_date(order='desc', limit=10)
txns[0].to_dict()

### Blocks

In [ ]:
# Transactions are grouped together into blocks, which are added sequentially to the chain
blocks = api.block.blocks_by_number(order='desc', limit=10)
blocks[0].to_dict()

In [ ]:
# Let's try retrieving all the transactions in this block
bn = 14980242
block_txns = api.block.transactions_by_block(block_number_above=bn, block_number_below=bn, limit=500)
print("Found {} txns for block {}".format(len(block_txns), bn))
block_txns[0].to_dict()

### Logs

In [ ]:
# Getting logs for a specific transaction
logs = api.block.logs_by_transaction(block_txns[0].transaction_hash)
logs[0].to_dict()

### Transactions and Blocks Summary

- This is similar data to what you'd get from an RPC API
- We provide it for completeness, but alone it's not easy to work with
- Consistent data models help

But working with blockchain data can be much easier.

## NFTs

- Retrieve info about a collection
- Quickly retrieve cached images and metadata from the CDN
- Query NFTs by owner
- Work with trait data

In [ ]:
# Let's check out an NFT Contract

bayc_contract_address = "0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D"
bayc = api.nft.collections_by_contract_address(bayc_contract_address)
bayc[0].to_dict()

In [ ]:
# Let's take a look at the collection's icon
from IPython.display import display

display(api.cdn.query(bayc[0].image_url).image())

In [ ]:
# Now let's take a look at the first 500 NFTs in the Bored Ape collection

nfts = api.nft.nfts_by_contract_address(bayc_contract_address, limit=500)
print("{} nfts retrieved\n".format(len(nfts)))
nfts[0].to_dict()

In [ ]:
img = api.cdn.query(nfts[5].image_url).image()
display(img)

## Wallet

In [ ]:
# Here's how we pull a wallet's nft balance

wallet = '0x3f4C85754A36Cb81067a0A88Bb1f83B9Cef97a91'
ens_address = "0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85"

all_nfts = api.nft.nfts_by_owner(wallet, limit=50)
nfts = api.nft.nfts_by_owner(wallet, ens_address, limit=50)


print("{} nfts retrieved".format(len(all_nfts)))
print("{} nfts retrieved when including only ENS NFTs".format(len(nfts)))

In [ ]:
nfts[1].to_dict()

In [ ]:
# Token data is accessible the same way
tokens = api.token.tokens_by_owner(wallet)
len(tokens)

### Full support ERC-1155 Semi Fungible tokens

Often used in gaming applications, these tokens include a balance.

In [ ]:
genesis_888 = '0x36d30B3b85255473D27dd0F7fD8F35e36a9d6F06'

owners = api.nft.owners_by_contract_address(genesis_888, limit=500)
owners[0].to_dict()

### All of a wallet's transfers are easily available

In [ ]:
# Pulling the wallet's activity is just as easy

api.nft.transfers_by_account(wallet)[:3]

In [ ]:
api.nft.transfers_by_account(wallet, limit=1)[0].to_dict()

### Mints are pulled the same way

In [ ]:
api.nft.transfers_by_account(wallet, transfer_category='mint', limit=1)[0].to_dict()

### Charting recent mints

In [ ]:
import pandas as pd
from IPython.display import Image
from transpose.extras import Plot
import time

t0 = time.time()
crypto_kitties_address = '0x06012c8cf97BEaD5deAe237070F9587f8E7A266d'

bulk_transfers = api.bulk_request(api.nft.transfers_by_contract_address(crypto_kitties_address, transfer_category='mint', limit=500, order='desc'), 20, 5000)
print("{} mints retrieved".format(len(bulk_transfers)))

# Aggregate by day
counts = {}
for transfer in bulk_transfers:
    if transfer.timestamp[:10] in counts.keys():
        counts[transfer.timestamp[:10]] += 1
    else:
        counts[transfer.timestamp[:10]] = 1
counts_list = [counts[x] for x in counts.keys()]
x = pd.date_range(bulk_transfers[0].timestamp, bulk_transfers[-1].timestamp, periods=len(counts_list))

# Set up chart
chart = Plot(title="Last 5000 mints")    
chart.add_data({
    'x': x,
    'y': counts_list,
    'y_axis': "Number of mints",
    'x_axis': "Time"
})

print("Took {}s".format(str(time.time() - t0)))
chart.render(path='/Users/mcalvey/Documents/transpose/code.nosync/live-demo/img/transfers.png', format='png')
Image(filename='/Users/mcalvey/Documents/transpose/code.nosync/live-demo/img/transfers.png')

## Fungible Tokens

### Let's dive into WETH

In [ ]:
# Let's pull top weth holders
weth_address = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"
top_holders = api.token.owners_by_contract_address(weth_address, limit=100)
balances = [x.balance for x in top_holders]
print("{} balances found".format(len(balances)))

In [ ]:
# How does this distribution look?
chart = Plot(title="WETH top 500 holder distribution")
chart.add_data({
    'x': list(range(len(balances))),
    'y': balances,
    'y_axis': 'Balance',
    'x_axis': 'Holder'
})
chart.render(path='/Users/mcalvey/Documents/transpose/code.nosync/live-demo/img/transfers.png', format='png')
Image(filename='/Users/mcalvey/Documents/transpose/code.nosync/live-demo/img/transfers.png')

In [ ]:
# Let's graph it
import datetime
import pandas as pd
from IPython.display import Image
from transpose.extras import Plot

# Get data
month_ago = datetime.datetime.now() - datetime.timedelta(days=31)
bulk_transfers = api.bulk_request(api.token.transfers_by_contract_address(weth_address, transferred_after=month_ago, limit=500), 20, 10000)

# Aggregate by day
counts = {}
for transfer in bulk_transfers:
    if transfer.timestamp[:10] in counts.keys():
        counts[transfer.timestamp[:10]] += 1
    else:
        counts[transfer.timestamp[:10]] = 1
counts_list = [counts[x] for x in counts.keys()]
x = pd.date_range(bulk_transfers[0].timestamp, bulk_transfers[-1].timestamp, periods=len(counts_list))

print(len(counts))
for i in counts.keys():
    print(i)
    print(counts[i])
    break
    
# Set up chart
chart = Plot(title="Last 100000 transfers")    
chart.add_data({
    'x': x,
    'y': counts_list,
    'y_axis': "Number of trades",
    'x_axis': "Time"
})

chart.render(path='/Users/mcalvey/Documents/transpose/code.nosync/live-demo/img/transfers.png', format='png')
Image(filename='/Users/mcalvey/Documents/transpose/code.nosync/live-demo/img/transfers.png')

## Extras

### Bored Ape Holder Analysis

In [ ]:
import time
# Time to dive deeper into holder data

# Get all holders of the NFT
t0 = time.time()
all_holders = api.bulk_request(api.nft.owners_by_contract_address(contract_address=bayc_contract_address, limit=500), 20, 10000)
t1 = time.time()
print("Took: {}".format(t1 - t0))
print("Total holders: " + str(len(all_holders)))

In [ ]:
# calculate how many BAYC tokens are held by each holder

number_owned_per_holder = {}
for holder in all_holders:
    if holder.owner in number_owned_per_holder: number_owned_per_holder[holder.owner] += 1
    else: number_owned_per_holder[holder.owner] = 1
sorted_holders = sorted(number_owned_per_holder.items(), key=lambda x: x[1], reverse=True)

top_holder = sorted_holders[0][0]
sorted_holders[:10]

In [ ]:
# Let's add ENS names
named_holders = {}
for holder in sorted_holders[:5]:
    name = api.ens.primary_ens_records_by_account(holder[0])
    if name != []:
        named_holders[name[0].ens_name] = holder[1]
    else:
        named_holders[holder[0]] = holder[1]
named_holders

In [ ]:
# Let's dive into what else the top 10 holders own

top_holders = sorted_holders[:5]
top_overlapping_contracts = {}

for holder in top_holders:
    print("Checking account address: " + holder[0])
    # get all the nfts owned by the holder
    all_nfts_owned = api.bulk_request(api.nft.nfts_by_owner(holder[0], limit=500))

    # get the contract address for each NFT and save the number of NFTs owned by each holder
    for nft in all_nfts_owned:
        if nft.contract_address in top_overlapping_contracts: top_overlapping_contracts[nft.contract_address] += 1
        else: top_overlapping_contracts[nft.contract_address] = 1

sorted_contract_addresses = sorted(top_overlapping_contracts.items(), key=lambda x: x[1], reverse=True)
sorted_contract_addresses[:10]

named_contracts = {}
for contract_address in sorted_contract_addresses[:10]:
    name = api.nft.collections_by_contract_address(contract_address[0])[0].name
    named_contracts[name] = contract_address[1]
named_contracts
